In [ ]:
import numpy as np
import tqdm
import json

In [ ]:
file = './21.txt'
lines = open(file, 'r').readlines()
lines = [l.strip('\n') for l in lines]


In [ ]:
info = [
    '593A',
    '283A',
    '670A',
    '459A',
    '279A',
    ]
poses = {
    'A':[2,1],
    'U':[1,1],
    'L':[0,2],
    'D':[1,2],
    'R':[2,2],
}


In [ ]:
from collections import Counter
mappings = {}
for curr in ['A','D','L','U','R']:
    mappings[curr] = {}
    for next in ['A','D','L','U','R']:
        moves_left = max(0, poses[curr][0]-poses[next][0])
        moves_right = max(0, poses[next][0]-poses[curr][0])
        moves_down = max(0, poses[next][1]-poses[curr][1])
        moves_up = max(0, poses[curr][1]-poses[next][1])
        if next!='L':
            # moves_button = ['U']*moves_up + ['R']*moves_right + ['D']*moves_down + ['L']*moves_left
            moves_button = ['R']*moves_right + ['L']*moves_left + ['U']*moves_up + ['D']*moves_down
        else:
            moves_button = ['R']*moves_right + ['D']*moves_down + ['L']*moves_left + ['U']*moves_up
        mappings[curr][next] = [''.join(moves_button + ['A'])]
        if not ((curr in ['A','U'] and next=='L') or (curr=='L' and next in ['A','U']) or len(Counter(moves_button))<=1):
            mappings[curr][next].append(''.join([l for l in reversed(moves_button)] + ['A']))

print(mappings)


In [ ]:
from functools import cache
@cache
def compute_moves_seq_len(seq, steps):
    prev = 'A'
    sub_sequences = []
    for el in seq:
        sub_sequences.append(mappings[prev][el])
        prev = el
    if steps==1:
        return sum([min([len(s) for s in ss]) for ss in sub_sequences])
    else:
        return sum([min([compute_moves_seq_len(s, steps-1) for s in ss]) for ss in sub_sequences])

def shorter_el(l):
    min_l = min([len(e) for e in l])
    el = [e for e in l if len(e)==min_l][-1]
    return el

@cache
def compute_moves_seq(seq, steps):
    prev = 'A'
    sub_sequences = []
    for el in seq:
        sub_sequences.append(mappings[prev][el])
        prev = el
    if steps==1:
        sub_sequences = [shorter_el([s for s in ss]) for ss in sub_sequences]
        return [s for ss in sub_sequences for s in ss]
    else:
        subs = [shorter_el([compute_moves_seq(s, steps-1) for s in ss]) for ss in sub_sequences]
        return [s for ss in subs for s in ss]


moves_to_do_codes = {
    '593A': [['LUUA','UULA'], ['RUA','URA'], ['DDA'], ['DA']],
    '283A': [['ULA','LUA'], ['UUA'], ['DDRA','RDDA'], ['DA']],
    '670A': [['UUA'], ['LLUA','ULLA'], ['RDDDA'], ['RA']],
    '459A': [['UULLA'], ['RA'], ['URA','RUA'], ['DDDA']],
    '279A': [['LUA','ULA'], ['LUUA','UULA'], ['RRA'], ['DDDA']],
}
num_keypads = 2

moves_to_do = {}
for code, moves in moves_to_do_codes.items():
    res = [shorter_el([compute_moves_seq(r, num_keypads) for r in rr]) for rr in moves]
    # for _ in range(num_keypads):
    #     res = [r for rr in res for r in rr]
    moves_to_do[code] = [r for rr in res for r in rr]

tot_res = 0
for code, flatten_moves in moves_to_do.items():
    n = int(code.strip('A'))
    str_move = ''.join(flatten_moves)
    code_moves = str_move.replace('D','v').replace('U','^').replace('L','<').replace('R','>')
    print(f'\nfor code={code}, n={n}, length={len(str_move)} res={n*len(str_move)}, \nstring moves : {str_move} \ncode moves : {code_moves}')
    tot_res += n*len(str_move)

num_moves_to_do = {}
for code, moves in moves_to_do_codes.items():
    resn = sum([min([compute_moves_seq_len(r, num_keypads) for r in rr]) for rr in moves])
    num_moves_to_do[code] = resn
tot_res2 = 0
for code, nmoves in num_moves_to_do.items():
    n = int(code.strip('A'))
    tot_res2 += n*nmoves
    print(f'code {code}, num moves to do : {nmoves}')

print(f'tot res : {tot_res}')
print(f'tot res2 : {tot_res2}')

In [ ]:
from functools import cache
@cache
def compute_moves_seq_len(seq, steps, keypads):
    prev = 'A'
    sub_sequences = []
    for el in seq:
        sub_sequences.append(mappings[prev][el])
        prev = el
    if steps==1:
        return sum([min([len(s) for s in ss]) for ss in sub_sequences])
    else:
        return sum([min([compute_moves_seq_len(s, steps-1, keypads) for s in ss]) for ss in sub_sequences])

moves_to_do_codes = {
    '593A': [['LUUA','UULA'], ['RUA','URA'], ['DDA'], ['DA']],
    '283A': [['ULA','LUA'], ['UUA'], ['DDRA','RDDA'], ['DA']],
    '670A': [['UUA'], ['LLUA','ULLA'], ['RDDDA'], ['RA']],
    '459A': [['UULLA'], ['RA'], ['URA','RUA'], ['DDDA']],
    '279A': [['LUA','ULA'], ['LUUA','UULA'], ['RRA'], ['DDDA']],
}
num_keypads = 25

num_moves_to_do = {}
for code, moves in moves_to_do_codes.items():
    # resn = sum([compute_moves_seq_len(r, num_keypads, num_keypads) for r in moves])
    resn = sum([min([compute_moves_seq_len(r, num_keypads, num_keypads) for r in rr]) for rr in moves])
    num_moves_to_do[code] = resn
tot_res2 = 0
for code, nmoves in num_moves_to_do.items():
    n = int(code.strip('A'))
    tot_res2 += n*nmoves
    print(f'code {code}, num moves to do : {nmoves}')

print(f'tot res2 : {tot_res2}')